In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from tqdm import tqdm

from utils.preprocessing import get_texts
from utils.preprocessing import get_texts, stop_words

In [2]:
# TODO: Modify here for different sectors and score_types
# set the sector and ESG type for the analysis
sector = "Technology"
# sector = "Energy"

# score_type = "governanceScore"
# score_type = "environmentScore"
score_type = "socialScore"

In [3]:
df_esg_score = pd.read_excel("data/esg_score.xlsx", sheet_name = "data")

In [4]:
df_esg_score["sector"].unique()

array(['Consumer Cyclical', 'Energy', 'Industrials', 'Healthcare',
       'Basic Materials', 'Consumer Defensive', 'Utilities', 'Technology',
       'Financial Services', 'Communication Services', 'Real Estate', nan],
      dtype=object)

In [5]:
tickers = df_esg_score[df_esg_score["sector"] == sector]["Company"]
esgs = df_esg_score[df_esg_score["sector"] == sector][["Company", "socialScore", "governanceScore", "environmentScore"]]

In [6]:
len(tickers)

63

In [7]:
score = esgs[score_type]

In [15]:
alpha = 0.15
upper_score = np.quantile(score, 1 - alpha)
lower_score = np.quantile(score, alpha)

In [16]:
bad_companies = esgs[esgs[score_type] > upper_score]["Company"].values
good_companies = esgs[esgs[score_type] < lower_score]["Company"].values

In [17]:
good_companies

array(['QCOM', 'MU', 'TRMB', 'NVDA', 'AMAT', 'WDC', 'IPGP', 'STX', 'KEYS',
       'CDW'], dtype=object)

In [18]:
bad_companies

array(['FTV', 'IT', 'VRSN', 'PTC', 'FTNT', 'FFIV', 'FISV', 'FIS', 'ORCL',
       'NLOK'], dtype=object)

In [19]:
esgs[esgs[score_type] > upper_score]

,Company,socialScore,governanceScore,environmentScore
22,FTV,14.18,5.53,15.63
177,IT,17.12,6.28,0.11
211,VRSN,12.87,6.28,3.04
259,PTC,12.72,5.66,1.24
260,FTNT,12.13,6.17,1.23
272,FFIV,12.49,5.40,1.26
292,FISV,10.73,6.41,1.19
295,FIS,11.14,5.83,1.27
303,ORCL,10.58,6.84,0.43
326,NLOK,10.55,4.95,1.17


In [20]:
bad_companies_score = esgs[esgs[score_type] > upper_score][score_type].values
good_companies_score = esgs[esgs[score_type] < lower_score][score_type].values

In [21]:
avg_bad = np.mean(bad_companies_score)
avg_good = np.mean(good_companies_score)
print(avg_bad, avg_good)

12.450999999999999 3.567


In [22]:
print(upper_score, lower_score)

10.514 4.295999999999999


In [23]:
ticker_library = pd.read_csv(os.path.join("data", "tickers.csv"))
good_cik = []
bad_cik = []
for ticker in good_companies:    
    try:
        # for a given ticker, find its cik number through th ticker library
        good_cik.append(ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:])
    except:
        # if could not find cik, give it a empty cik
        good_cik.append('')

for ticker in bad_companies:    
    try:
        # for a given ticker, find its cik number through th ticker library
        bad_cik.append(ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:])
    except:
        # if could not find cik, give it a empty cik
        bad_cik.append('')

/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
ret_good = get_texts(good_cik, good_companies)

0it [00:00, ?it/s]

Scraping CIK 0000723125


100%|██████████| 1/1 [00:07<00:00,  7.15s/it]


Scraping CIK 0000723125


100%|██████████| 1/1 [00:09<00:00,  9.77s/it]
2it [00:29, 14.52s/it]

...

100%|██████████| 1/1 [00:09<00:00,  9.78s/it]
5it [00:52,  9.17s/it]

Scraping CIK 0000106040


100%|██████████| 1/1 [00:07<00:00,  7.50s/it]


Scraping CIK 0000106040


100%|██████████| 1/1 [00:10<00:00, 10.57s/it]
10it [01:23,  8.39s/it]


In [25]:
ret_bad = get_texts(bad_cik, bad_companies)

5it [00:00, 14.45it/s]

Scraping CIK 0000798354


100%|██████████| 1/1 [00:07<00:00,  7.43s/it]
5it [00:14, 14.45it/s]

Scraping CIK 0000798354


100%|██████████| 1/1 [00:11<00:00, 11.09s/it]
7it [00:33,  6.50s/it]

...

100%|██████████| 1/1 [00:10<00:00, 10.55s/it]
9it [01:02,  9.49s/it]

Scraping CIK 0000849399


100%|██████████| 1/1 [00:07<00:00,  7.83s/it]


Scraping CIK 0000849399


100%|██████████| 1/1 [00:09<00:00,  9.85s/it]
10it [01:28,  8.84s/it]


In [26]:
good_docs = ret_good["docs"]
bad_docs = ret_bad["docs"]

In [34]:
# TODO: Modify here for different ngram range
n_min = 2
n_max = 5
cv = CountVectorizer(max_df=0.7, stop_words=stop_words, max_features=200, ngram_range=(n_min, n_max))
word_count_vector = cv.fit_transform(good_docs + bad_docs)

/Users/luckywang/Documents/Document/Course Material/Fall 2021/esg_nlp/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['10'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [35]:
feature_names = cv.get_feature_names()

In [36]:
count_feature = word_count_vector.toarray().sum(axis=0)

In [37]:
feature_names = cv.get_feature_names()

In [38]:
d = {"word": [], "good_score": [], "bad_score": [], "good_score_all": []
    , "bad_score_all": [], "count": [], "good_nums": [], "bad_nums": []}

for feature_idx, word in enumerate(feature_names):
    good_sum = bad_sum = good_num = bad_num = 0

    for i, doc_set in enumerate(good_docs):
        if word in doc_set:
            good_num += 1
            good_sum += good_companies_score[i]
    for i, doc_set in enumerate(bad_docs):
        if word in doc_set:
            bad_num += 1
            bad_sum += bad_companies_score[i]
    
    # print("word: {}".format(word))
    d["word"].append(word) 
    
    if good_num:
        d["good_score"].append(good_sum / good_num)
    else:
        d["good_score"].append(0)
    if bad_num:
        d["bad_score"].append(bad_sum / bad_num)
    else:
        d["bad_score"].append(0)

    d["good_score_all"].append(good_sum / len(good_docs))
    d["bad_score_all"].append(bad_sum / len(bad_docs))

    d["count"].append(count_feature[feature_idx])
    d["good_nums"].append(good_num)
    d["bad_nums"].append(bad_num)


In [39]:
df = pd.DataFrame(data=d)


In [40]:
df.head()

,word,good_score,bad_score,good_score_all,bad_score_all,count,good_nums,bad_nums
0,acquired intangible,3.731667,12.43875,2.239,9.951,259,6,8
1,acquisition first,3.435000,11.63000,0.687,4.652,259,2,4
2,additional information regarding,3.752000,12.54000,1.876,8.778,303,5,7
3,adverse effect result financial,3.433333,12.78000,2.060,5.112,408,6,4
4,aggregate principal amount senior,3.597500,12.68000,1.439,10.144,260,4,8


In [41]:
df["diff"] = abs(df["good_nums"] - df["bad_nums"])

In [42]:
df = df.sort_values("diff", ascending=False)#.head(60)

In [43]:
df.head(20)

,word,good_score,bad_score,good_score_all,bad_score_all,count,good_nums,bad_nums,diff
98,integrated circuit,3.582500,11.635000,2.866,2.327,515,8,2,6
45,deferred revenue,3.397500,12.451000,1.359,12.451,596,4,10,6
62,ended october,3.607500,10.640000,2.886,2.128,329,8,2,6
183,support service,3.370000,11.932222,1.011,10.739,340,3,9,6
102,june compared,3.405000,13.235714,0.681,9.265,265,2,7,5
57,ended january,3.607500,13.386667,2.886,4.016,823,8,3,5
60,ended june compared,3.405000,13.235714,0.681,9.265,251,2,7,5
126,month period,0.000000,12.252000,0.000,6.126,462,0,5,5
161,report year,3.790000,12.662222,1.516,11.396,228,4,9,5
162,report year ended,3.790000,12.662222,1.516,11.396,228,4,9,5


In [44]:
goodvbad_path = os.path.join("data", "goodvbad")
if not os.path.isdir(goodvbad_path):
    os.mkdir(goodvbad_path)

In [45]:
df.round(2).to_csv("data/goodvbad/{}_{}_{}_n{}-{}.csv".format(sector[:8], score_type[:3], alpha, n_min, n_max))

## Appendix

In [ ]:
# To remove all the previous files earlier than 2020
for cik in tqdm(bad_cik):
    tenk_path = os.path.join("data", "10k", cik, "rawtext")
    all_raws = os.listdir(tenk_path)
    for filename in all_raws:
        if filename[0] == '.':
            continue
        year = int(filename.split('_')[1].split('-')[0])
        # print(year)
        if year < 2020:
            # print(os.path.join(tenk_path, filename))
            os.remove(os.path.join(tenk_path, filename))
    
    tenq_path = os.path.join("data", "10q", cik, "rawtext")
    all_raws = os.listdir(tenq_path)
    for filename in all_raws:
        if filename[0] == '.':
            continue
        year = int(filename.split('_')[1].split('-')[0])
        # print(year)
        if year < 2020:
            # print(os.path.join(tenk_path, filename))
            os.remove(os.path.join(tenq_path, filename))


In [4]:
df = pd.read_csv("energy_good_vs_bad_uni_bi_tri.csv", index_col=0)

In [5]:
df.head()

,word,good_score,bad_score,good_score_all,bad_score_all,good_nums,bad_nums,diff
0,accompanying note consolidated,9.773333,23.7600,4.886667,3.960000,3,1,2
1,accompanying note integral,10.557500,21.6950,7.038333,7.231667,4,2,2
2,acmp,0.000000,21.9800,0.000000,3.663333,0,1,1
3,activity cash flow,10.975000,21.0660,7.316667,17.555000,4,5,1
4,adjusted ebitda,0.000000,22.2825,0.000000,14.855000,0,4,4


In [237]:
comp = pd.read_csv("data/sp500_component_stocks.csv")


In [241]:
comp[comp['A'] == "C"]["Agilent Technologies Inc."]

79    Citigroup Inc.
Name: Agilent Technologies Inc., dtype: object

In [140]:

comp[comp['A'].isin(good_companies)]["Agilent Technologies Inc."]

44                   Apache Corporation
111         Cabot Oil & Gas Corporation
128                 Chevron Corporation
163                  EOG Resources Inc.
322            Marathon Oil Corporation
358    Occidental Petroleum Corporation
Name: Agilent Technologies Inc., dtype: object

In [141]:
comp[comp['A'].isin(bad_companies)]["Agilent Technologies Inc."]

215            Halliburton Company
274      Kinder Morgan Inc Class P
343    National Oilwell Varco Inc.
354                     ONEOK Inc.
411                Schlumberger NV
487        Williams Companies Inc.
Name: Agilent Technologies Inc., dtype: object